# 화자 분리 모델 설명
- 정의 및 특징
    - Speaker Diarization
    - Whisper가 지원하지 않는 화자 분리 기능을 보완하기 위해 별도로 사용
- 사용 방법
    - 모델 제공: Hugging Face (pyannote/speaker-diarization-3.1)
    - 접근 권한: Model card의 Requirements 동의 및 만족 후 read Access Token 발급 필요
- RTTM 파일
    - 화자 분리 결과가 저장되는 표준 파일 형식 (Rich Transcription Time Marked)
    - 각 화자가 언제(시작 시간, 지속 시간) 말했는지에 대한 정보 포함

In [ ]:
%pip install pyannote.audio
%pip install numpy==1.26

  Using cached numpy-2.4.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (6.6 kB)
Using cached numpy-2.4.1-cp312-cp312-macosx_14_0_arm64.whl (5.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-1.26.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (53 kB)
Using cached numpy-1.26.0-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.4.1
    Uninstalling numpy-2.4.1:
      Successfully uninstalled numpy-2.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyannote-core 6.0.1 requires numpy>=2.0, but you have numpy 1.26.0 which is incompatible.
pyannote-metrics 4.0.0 requires numpy>=2.2.2, but you have numpy 1.26.0 wh

## 화자 분리 모델 실습: Pipeline 구성
- Apple Silicon 하드웨어 가속
    - MPS(Metal Performance Shaders): Apple GPU를 활용하기 위한 가속 프레임워크
    - 특징: NVIDIA의 CUDA 대신 mps 장치를 지정하여 Mac에서 모델 실행 속도를 10~20배 가속 가능
    - 설정: device="mps" 옵션으로 사용 (macOS 12.3 이상 권장)

In [ ]:
import os
from dotenv import load_dotenv

# Hugging Face 토큰 로드
load_dotenv()
HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

# 다이어리제이션 파이프라인 로드
from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  token=HUGGING_FACE_TOKEN
)

import torch

# 가용 디바이스 확인 및 적용
if torch.cuda.is_available():
    print('cuda is available')
else:
    print('cuda is not available')

if torch.backends.mps.is_available():
    pipeline.to(torch.device("mps"))
    print("mps is available")
else:
    print("mps is not available")

cuda is not available
mps is available


## 화자 분리 모델 실습: 화자 분리 후 RTTM 저장


In [ ]:
import torchaudio

# 오디오 로드 후 다이어리제이션 수행
waveform, sr = torchaudio.load("../audio/싼기타_비싼기타.mp3")
out = pipeline({"waveform": waveform, "sample_rate": sr})

# RTTM 형식으로 결과 저장
ann = out.speaker_diarization
with open("싼기타_비싼기타.rttm", "w", encoding="utf-8") as rttm:
    ann.write_rttm(rttm)

/Users/donggyeong/develop/now/GPT_AGENT_2025_BOOK/venv/lib/python3.12/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(


## 화자 분리 모델 실습: RTTM -> DataFrame -> CSV 정리


In [ ]:
import pandas as pd
rttm_path = "./싼기타_비싼기타.rttm"

# RTTM 파일을 DataFrame으로 파싱
df_rttm = pd.read_csv(
    rttm_path,
    sep=' ',
    header=None,
    names=['type', 'file', 'chnl', 'start', 'duration', 'C1', 'C2', 'speaker_id', 'C3', 'C4']
)

# start + duration을 end로 변환
df_rttm['end'] = df_rttm['start'] + df_rttm['duration']

# 연속된 동일 화자를 하나의 구간 번호로 묶기
df_rttm["number"] = None
df_rttm.at[0, "number"] = 0

for i in range(1, len(df_rttm)):
    if df_rttm.at[i, "speaker_id"] != df_rttm.at[i-1, "speaker_id"]:
        df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"] + 1
    else:
        df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"]

# 구간 번호 기준으로 시작/끝/화자 집계
df_rttm_grouped = df_rttm.groupby("number").agg(
    start=pd.NamedAgg(column='start', aggfunc='min'),
    end=pd.NamedAgg(column='end', aggfunc='max'),
    speaker_id=pd.NamedAgg(column='speaker_id', aggfunc='first')
)

# 구간 길이 계산 후 CSV 저장
df_rttm_grouped["duration"] = df_rttm_grouped["end"] - df_rttm_grouped["start"]
df_rttm_grouped = df_rttm_grouped.reset_index(drop=True)
display(df_rttm_grouped)

df_rttm_grouped.to_csv(
    "./싼기타_비싼기타_rttm.csv",
    sep=',',
    index=False
)

,start,end,speaker_id,duration
0,0.993,30.204,SPEAKER_00,29.211
1,32.414,41.391,SPEAKER_01,8.977
2,41.611,42.455,SPEAKER_00,0.844
3,41.645,42.708,SPEAKER_01,1.063
4,42.674,44.024,SPEAKER_00,1.350
5,45.813,67.109,SPEAKER_01,21.296
6,67.227,82.786,SPEAKER_00,15.559
7,84.659,102.564,SPEAKER_01,17.905
8,103.492,117.532,SPEAKER_00,14.040
9,119.759,138.676,SPEAKER_01,18.917
